## Optimization for the win

In this notebook, we will use formal constained management optimization as a replacement for boring 'ole scenarios.  Buckle up!

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import numpy as np
import pandas as pd
font = {'size'   : 10}
import matplotlib
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt;
import shutil
import psutil

import sys
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd


In [ ]:
# specify the temporary working folder
t_d = os.path.join('pst_template_opt')
if os.path.exists(t_d):
    shutil.rmtree(t_d)

org_t_d = os.path.join("master_ies0")
if not os.path.exists(org_t_d):
    raise Exception()

shutil.copytree(org_t_d,t_d)

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"pest.pst"))
pe = pst.ies.get("paren",pst.ies.phiactual.iteration.max())

We want to use the "calibrated" model, which is the "base" realziation from IES:

In [ ]:
par = pst.parameter_data
par.loc[pe.columns,"parval1"] = pe.loc["base",:].values.flatten()

In [ ]:
pst.control_data.noptmax = 0
pst.write(os.path.join(t_d,"pest.pst"),version=2)
pyemu.os_utils.run("pestpp-ies pest.pst",cwd=t_d)

In [ ]:
pst.set_res(os.path.join(t_d,"pest.base.rei"))
obs = pst.observation_data

Let's plot the calibrated HK array and the end-of-mining heads

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws=t_d)
gwf = sim.get_model()
X,Y = gwf.modelgrid.xcellcenters,gwf.modelgrid.ycellcenters

In [ ]:
hkobs = obs.loc[obs.obsnme.str.contains("hk"),:].copy()
hkobs["i"] = hkobs.i.astype(int)
hkobs["j"] = hkobs.j.astype(int)
hkarr = np.zeros((hkobs.i.max()+1,hkobs.j.max()+1))
hkarr[hkobs.i,hkobs.j] = pst.res.loc[hkobs.obsnme,"modelled"].values

In [ ]:
eomhdsobs = obs.loc[(obs.obsnme.str.contains("hdslay1_t2")),:].copy()
eomhdsobs["i"] = eomhdsobs.i.astype(int)
eomhdsobs["j"] = eomhdsobs.j.astype(int)
eomarr = np.zeros((eomhdsobs.i.max()+1,eomhdsobs.j.max()+1))
eomarr[eomhdsobs.i,eomhdsobs.j] = pst.res.loc[eomhdsobs.obsnme,"modelled"].values
eomhdsobs

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,6))
ax.pcolormesh(X,Y,hkarr)
lb = ax.contour(X,Y,eomarr,levels=10,colors="w")
_= ax.clabel(lb)

In [ ]:
forecasts = pst.pestpp_options["forecasts"].split(",")
forecasts.sort()
forecasts

In [ ]:
res = pst.res
res.loc[forecasts,:]

In [ ]:
res.loc[res.name.str.contains("gde"),:]

So our calibrated model is over pumping: the simulated pit level is lower than 80 and the end of mining GDE flux is not even close to the pre-development estimate...

Let's see if we can optimize our way out of this issue.  First we need to define our optimization problem in terms of decision variables, constraints, and the objective function

In [ ]:

forecasts

Get the name of dewatering and reinjection wells and set some bounds.  For fun, lets allow every well to be either an extractor or injector

In [ ]:
wpar = par.loc[par.parnme.str.contains("wel"),:]

In [ ]:
wpar.shape
mpar = wpar.loc[wpar.parnme.str.contains("mar"),:]
dpar = wpar.loc[wpar.parnme.str.contains("dewater"),:]


In [ ]:
par.loc[mpar.parnme,"partrans"] = "none"
par.loc[mpar.parnme,"parlbnd"] = 0
par.loc[mpar.parnme,"parubnd"] = 1500
par.loc[dpar.parnme,"partrans"] = "none"
par.loc[dpar.parnme,"parlbnd"] = -3000
par.loc[dpar.parnme,"parubnd"] = 0

pst.pestpp_options["opt_dec_var_groups"] = wpar.pargp.unique().tolist()
pargp = pst.parameter_groups
pargp.loc[mpar.pargp.iloc[0],"inctyp"] = "absolute"
pargp.loc[mpar.pargp.iloc[0],"derinc"] = 500
pargp.loc[dpar.pargp.iloc[0],"inctyp"] = "absolute"
pargp.loc[dpar.pargp.iloc[0],"derinc"] = -1000


Now for constraints.  First, we want to make sure that whatever happens, we dont simulate water levels above land surface at well locations

In [ ]:
obs = pst.observation_data
hobs = obs.loc[obs.obsnme.str.contains("hdslay1_t2"),:].copy()
assert hobs.shape[0] > 0
hobs["i"] = hobs.i.astype(int)
hobs["j"] = hobs.j.astype(int)
top = np.loadtxt(os.path.join(t_d,"model.dis_top.txt")).reshape(hobs.i.max()+1,hobs.j.max()+1)

wpar['i'] = wpar.idx1.astype(int)
wpar['j'] = wpar.idx2.astype(int)
for i,j in zip(wpar.i,wpar.j):
    ijhobs = hobs.loc[(hobs.i==i) & (hobs.j==j),:]
    assert ijhobs.shape[0] == 1
    obs.loc[ijhobs.obsnme,"weight"] = 1
    obs.loc[ijhobs.obsnme,"obgnme"] = "less_than"
    obs.loc[ijhobs.obsnme,"obsval"] = top[i,j]    

Now we want to try to maintain the end-of-mining flux to the GDE to be comparable to what it was pre-development

In [ ]:
predev_gde_val = obs.loc[forecasts[0].replace("time:3651","time:1"),"obsval"]
print(predev_gde_val)
rhs_val = predev_gde_val
obs.loc[forecasts[0],"obsval"] = rhs_val
obs.loc[forecasts[0],"obgnme"] = "less_than"
obs.loc[forecasts[0],"weight"] = 1.0

And we want the water level in the pit to be less than 80

In [ ]:
obs.loc[forecasts[-1],"obsval"] = 80
obs.loc[forecasts[-1],"obgnme"] = "less_than"
obs.loc[forecasts[-1],"weight"] = 1.0

Make sure we arent reinjecting more water than we are extracting

In [ ]:
diffobs = obs.loc[(obs.oname=="inc") & (obs.usecol=="totwel") & (obs.totim=="3651"),:]
assert diffobs.shape[0] == 1
obs.loc[diffobs.obsnme,"weight"] = 1.0
obs.loc[diffobs.obsnme,"obsval"] = 0.0
obs.loc[diffobs.obsnme,"obgnme"] = "less_than"

For an objective function, we want to minimize the total dewatering flux because it costs money!

In [ ]:
aobs = obs.loc[(obs.oname=="inc") & (obs.usecol=="wel") & (obs.totim=="3651"),:]
assert aobs.shape[0] == 1
objname = aobs.obsnme.values[0]
#obs.loc[objname,"weight"] = 1.0
obs.loc[objname,"obgnme"] = "greater_than"
pst.pestpp_options["opt_obj_func"] = objname
pst.pestpp_options["opt_direction"] = "max"

And just to make sure MF6 isnt cheating, we want to make sure the auto-flow-reduce isnt happening

In [ ]:
wrobs = obs.loc[obs.obsnme.str.contains("wel-reduc"),:]
assert wrobs.shape[0]> 0
obs.loc[wrobs.obsnme,"weight"] = 1
obs.loc[wrobs.obsnme,"obgnme"] = "greater_than"
obs.loc[wrobs.obsnme,"obsval"] = -1

Thats it!  Lets run it...

In [ ]:
pst.control_data.noptmax = 1

In [ ]:
pst.write(os.path.join(t_d,"pest.pst"),version=2)

In [ ]:
num_workers = 10
m_d = "master_opt0"

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-opt', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

In [ ]:
jcb = pyemu.Matrix.from_binary(os.path.join(m_d,"pest.1.jcb")).to_dataframe()
jcb.loc[forecasts,:].T

Check your understanding:  What are those numbers on the response matrix?  What percent of the water injected at a given `mar` well ends up in the GDE?

In [ ]:
pst.set_res(os.path.join(m_d,"pest.res"))
res = pst.res
res.loc[res.name.str.contains("gde"),:]

In [ ]:
pst.parrep(os.path.join(m_d,"pest.{0}.par".format(pst.control_data.noptmax)))

In [ ]:
par = pst.parameter_data
par.loc[wpar.parnme,:]

In [ ]:
wpar["x"] = X[wpar.i,wpar.j]
wpar["y"] = Y[wpar.i,wpar.j]
eomarr[eomhdsobs.i,eomhdsobs.j] = pst.res.loc[eomhdsobs.obsnme,"modelled"]
fig,ax = plt.subplots(1,1)
ax.pcolormesh(X,Y,hkarr)
ax.scatter(wpar.x,wpar.y,marker='.',s=80,c=par.loc[wpar.parnme,"parval1"].values,cmap="coolwarm")
lb = ax.contour(X,Y,eomarr,levels=10,colors='w')
ax.clabel(lb)


Interesting pattern hey?!  



Lets checkout the optimal objective funtion:

In [ ]:
dfobj = pd.read_csv(os.path.join(m_d,"pest.slp.iobj.csv"))
dfobj

## Reusing the response matrix

We can now reuse that response matrix and change any number of inputs to the optimization problem...


In [ ]:
resp_mat_fname = "pest.1.jcb"
assert os.path.exists(os.path.join(m_d,resp_mat_fname))
pst.pestpp_options["base_jacobian"] = resp_mat_fname

Let's increase the allowable upper bound on the reinjection wells, which greatly increases the feasible space...  

In [ ]:
pst.parameter_data.loc[mpar.parnme,"parubnd"] = 3500

In [ ]:
pst.control_data.noptmax = 1
pst.write(os.path.join(m_d,"pest.pst"),version=2)
pyemu.os_utils.run("pestpp-opt pest.pst",cwd=m_d)


In [ ]:
pst.parrep(os.path.join(m_d,"pest.{0}.par".format(pst.control_data.noptmax)))
par = pst.parameter_data
pst.set_res(os.path.join(m_d,"pest.res"))
res = pst.res
res.loc[res.name.str.contains("gde"),:]

In [ ]:
eomarr[eomhdsobs.i,eomhdsobs.j] = pst.res.loc[eomhdsobs.obsnme,"modelled"]
fig,ax = plt.subplots(1,1)
ax.pcolormesh(X,Y,hkarr)
ax.scatter(wpar.x,wpar.y,marker='.',s=80,c=par.loc[wpar.parnme,"parval1"].values,cmap="coolwarm")
lb = ax.contour(X,Y,eomarr,levels=10,colors='w')
ax.clabel(lb)

## Reliability

Lets see how the posterior ensemble of realizations does in terms of feasibility

In [ ]:
forecasts

In [ ]:
oe_pt = pst.ies.get("obsen",pst.ies.phiactual.iteration.max())
oe_feas = oe_pt.loc[oe_pt[forecasts[-1]]<80,:]
oe_feas = oe_feas.loc[oe_feas[forecasts[0]]<rhs_val,:]
oe_feas.loc[:,forecasts]

So only a few realizations are feasible...we can calculate the "reliability" on the base scenario (with the standard rates)

In [ ]:
reliability = 100 * oe_feas.shape[0]/oe_pt.shape[0]
print(reliability," percent reliable")

yikes!  We probably dont want to go to press with these results...

Let see if we can use the ies ensemble within the optimizer to find a more reliable solution...

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"pest.pst"))
pst.pestpp_options

In [ ]:
pst.pestpp_options["opt_par_stack"] = "pest.{0}.par.jcb".format(pst.ies.phiactual.iteration.max())
pst.pestpp_options["opt_obs_stack"] = "pest.{0}.obs.jcb".format(pst.ies.phiactual.iteration.max())
pst.pestpp_options["opt_recalc_chance_every"] = 9999
pst.pestpp_options["opt_risk"] = 0.95 #this is the fractional reliability value - bigger is more conservative...
pst.control_data.noptmax = 1
pst.write(os.path.join(m_d,"pest.pst"),version=2)
#m_d = "master_opt_reliable"

In [ ]:
pyemu.os_utils.run("pestpp-opt pest.pst",cwd=m_d)

In [ ]:
pst.parrep(os.path.join(m_d,"pest.{0}.par".format(pst.control_data.noptmax)))
par = pst.parameter_data
pst.set_res(os.path.join(m_d,"pest.res"))

In [ ]:
par.loc[wpar.parnme,:]

In [ ]:
eomarr[eomhdsobs.i,eomhdsobs.j] = pst.res.loc[eomhdsobs.obsnme,"modelled"]
fig,ax = plt.subplots(1,1)
ax.pcolormesh(X,Y,hkarr)
ax.scatter(wpar.x,wpar.y,marker='.',s=80,c=par.loc[wpar.parnme,"parval1"].values,cmap="coolwarm")
lb = ax.contour(X,Y,eomarr,levels=10,colors='w')
ax.clabel(lb)

Wait, we only need to get down to 80 in the pit for dewatering, why is it less than 70 now?!  

That is the "cost" of uncertainty: to ensure that the decision support we provide is highly reliable in the face of uncertainty, we have to design an system that "over" dewaters and "over" reinjects.  The optimal design we found IS highly reliable with respect to expected uncertainties in hydraulic properties and some boundary condition representations